# Setting

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn, einsum
import torch.nn.functional as F
from perceiver_pytorch.perceiver_pytorch import exists, default, cache_fn, fourier_encode, PreNorm, FeeodForward, Attention

# Import Data

In [ ]:
MODALITY = 'IMAGE'   # IMAGE/TEXT. 모드 바꾸면 학습 모달리티 바뀜

TUNING_MODE = 'OFF'  # Hyperparam tuning 모드. 차후 구현

In [ ]:
if MODALITY == 'IMAGE':
    X_train_np, y_train, X_test, y_test = text_loader()
else:
    X_train_np, y_train, X_test, y_test = language_loader()

# Import Model

# Train

# Train

## Learning Curve

# Load model